## Métrica quejas

In [693]:
#ver lo de generenias para el futuro

In [1]:
from dbconn import querydb, querydbtopandas 
import pandas as pd 
import datetime
import calendar

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
menor = str(datetime.datetime.today() - datetime.timedelta(days=20))[:7]

mayor = str(datetime.datetime.today())[:7] # - datetime.timedelta(days=20))[:7]
gerencia = "18, 19"

In [4]:
gerencias = querydbtopandas(f"""select id_gerencia,ger_Descripcion from DBA.CRM_GERENCIAS """)

In [6]:
mayor= "2020-05"
menor = "2020-04"

In [7]:
data = querydbtopandas(f"""select a11.workflow_id  workflow_id,
   max(a113.detalle)  detalle3,
  a12.prestad  id_pre_prestador,
  a18.GERENCIA,
  a11.registro_fecha_max,
   max(a13.desc_pre_nombre)  desc_pre_nombre,
   sum(1.0)  CANTIDADETAPASMINNDISTINCT
   from   DBA.CRM_ETAPAS_MIN_MAX a11
   join   DBA.crm_quejas_categ_resol a12
     on   (a11.etapa_id_max = a12.ETAPA_ID)
  join   DBA.d_pre_prestador    a13
     on   (a12.prestad = a13.id_pre_prestador)         
   join   DBA.crm_atributo_quejas    a113
     on   (a12.atributo_id = a113.atributo_id)  
   join   DBA.ft_pad_padron_iq   a14
     on   (a11.ID_AFI_AFILIADO = a14.id_afiliados and
   a11.id_tie_mes_max = a14.MES)  
   join   DBA.CRM_SUBMOTIVOS a18
     on   (a12.SUBMOTIVO_ID = a18.SUBMOTIVO_ID and
   a12.motivo_llamado_id = a18.motivo_id)
where  (a11.registro_fecha_max BETWEEN date("{menor}") and date("{mayor}")
and a11.motivo_id = 8
and a12.estado in (14)
and a18.GERENCIA in ({gerencia})
and a12.prestad <> 0)
group by   a11.workflow_id,
    a11.registro_fecha_max,
   a18.GERENCIA,
   a12.finalizado,
   prestad,
   a12.ALTA_FECHA
   """)

In [8]:
data.registro_fecha_max = data.registro_fecha_max.astype(str).str[:7]

In [10]:
data.registro_fecha_max.unique()

array(['2020-04'], dtype=object)

In [42]:
for i in data.registro_fecha_max.unique():
    
    #tope de quejas no esta aplicado 
    
    df = data[data["registro_fecha_max"] == i]
    df_metrica = df[["workflow_id", "detalle3", "id_pre_prestador","desc_pre_nombre" ]]

    final = df_metrica.groupby([ "id_pre_prestador","desc_pre_nombre", "detalle3" ])["workflow_id"].nunique().reset_index()

    total_prestador = final.groupby([ "id_pre_prestador","desc_pre_nombre"])["workflow_id"].sum().rename("total_prestador")

    final = final.merge(total_prestador, left_on = "id_pre_prestador" , right_on="id_pre_prestador" )

    final = final.rename(columns= {"id_pre_prestador": "Prestador", "desc_pre_nombre": "Razón social", "detalle3": "Clasificación de casos", "workflow_id": "Q de quejas", "total_prestador": "Total de quejas prestador"})
    final.to_parquet(f'quejas_datos/{i}.gzip',compression='gzip') 